# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [7]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 13
generator = square_root_generator(limit)

sum = 0
for sqrt_value in generator:
    # print(sqrt_value)
    sum += sqrt_value
    print(sum)


hello world


# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [ ]:
!pip install dlt[duckdb]

import dlt
import duckdb

p1_data = []
p2_data = []

def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

sum = 0
for person in people_1():
    sum += person['Age']
    p1_data.append(person)

pipeline = dlt.pipeline(pipeline_name='people_data',destination='duckdb', dataset_name='people')

p1_info = pipeline.run(p1_data, table_name='people', write_disposition='replace')

conn = duckdb.connect(f"{pipeline.pipeline_name}.duckdb")
conn.sql(f"SET search_path = '{pipeline.dataset_name}'")


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    sum += person['Age']
    p2_data.append(person)

p2_info = pipeline.run(p2_data, table_name='people', write_disposition='append')
persons = conn.sql("SELECT * FROM people").df()
display(persons)

,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1707180691.393217,Gr0KkvxiAGRN3Q,None
1,2,Person_2,27,City_A,1707180691.393217,F/FFUSIHyvo7gA,None
2,3,Person_3,28,City_A,1707180691.393217,4Ak5fagI3LkAOQ,None
3,4,Person_4,29,City_A,1707180691.393217,EMRRMXDpNIFgvA,None
4,5,Person_5,30,City_A,1707180691.393217,HvVgnOVtXaYmlA,None
5,3,Person_3,33,City_B,1707180691.9752169,nY1wD0HzOVUp/g,Job_3
6,4,Person_4,34,City_B,1707180691.9752169,Yn/x162YiZZ3/Q,Job_4
7,5,Person_5,35,City_B,1707180691.9752169,WtGNi+I4Tw8R6g,Job_5
8,6,Person_6,36,City_B,1707180691.9752169,p6ueTFId6Qho8Q,Job_6
9,7,Person_7,37,City_B,1707180691.9752169,GxKIzkPtFQArqQ,Job_7


# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


# Solution: First make sure that the following modules are installed:

In [2]:
#Install the dependencies
# %%capture
# !pip install dlt[duckdb]

import dlt
import duckdb

p1_data = []
p2_data = []

def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

sum = 0
for person in people_1():
    sum += person['Age']
    p1_data.append(person)

pipeline = dlt.pipeline(pipeline_name='people_data',destination='duckdb', dataset_name='people')

p1_info = pipeline.run(p1_data, table_name='people_merge', write_disposition='replace')

conn = duckdb.connect(f"{pipeline.pipeline_name}.duckdb")
conn.sql(f"SET search_path = '{pipeline.dataset_name}'")

def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    sum += person['Age']
    # print(sum)
    p2_data.append(person)

# Add people_2 & merge on primary_key
p2_info = pipeline.run(p2_data, table_name='people_merge', write_disposition='merge', primary_key='id')

persons_merge = conn.sql("SELECT * FROM people.people_merge").df()
display(persons_merge)

# Get sum of added people
sum = conn.sql("SELECT SUM(age) FROM people.people_merge")
display(sum)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 1.1 MB/s eta 0:00:00ta 0:00:01
  Using cached wheel-0.42.0-py3-none-any.whl.metadata (2.2 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 47.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.8/126.8 kB 3.3 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 540.2/540.2 kB 14.5 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.3/51.3 kB 1.5 MB/s eta 0:00:00
Using cached wheel-0.42.0-py3-none-any.whl (65 kB)

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1707433664.350885,RHg6X0hQpJD6KA,None
1,2,Person_2,27,City_A,1707433664.350885,oRgLYsDsEg3iow,None
2,5,Person_5,35,City_B,1707433665.6351168,iwsGNe48g4UDdQ,Job_5
3,8,Person_8,38,City_B,1707433665.6351168,26rxK2Ry4vvSxQ,Job_8
4,7,Person_7,37,City_B,1707433665.6351168,d7JbuONzD4k3hQ,Job_7
5,4,Person_4,34,City_B,1707433665.6351168,wXtzog+yynuQtg,Job_4
6,3,Person_3,33,City_B,1707433665.6351168,hKzI9axirIeaqA,Job_3
7,6,Person_6,36,City_B,1707433665.6351168,NvpsatmOVX2RSA,Job_6


┌──────────┐
│ sum(age) │
│  int128  │
├──────────┤
│      266 │
└──────────┘

Questions? difficulties? We are here to help.
- DTC data engineering course channel: https://datatalks-club.slack.com/archives/C01FABYF2RG
- dlt's DTC cohort channel: https://dlthub-community.slack.com/archives/C06GAEX2VNX